In [13]:
import pandas as pd
import numpy as np
import math
import re
from tkinter import *
from pandastable import Table, TableModel


def generate_table(df):
    root = Tk()
    root.geometry("600x500")
    root.title("Absolute Linker")
    frame = Frame(root)
    frame.pack(fill="both", expand=True)
    table = Table(frame, dataframe=df, width=500, maxcellwidth=30)
    table.show()
    table.showindex = True
    root.mainloop()


def AbsoluteLoader():
    # Parsing Input File from user
    with open('./input.txt') as my_file:
        HTE = [line.split('.') for line in my_file]
    TRecord = HTE[1:-1]  # Extracting TRecord
    Programlength = HTE[0][3]
    StartingAddress = HTE[0][2]
    EndingAddress = hex(int(Programlength, 16) + int(StartingAddress, 16))[2:].zfill(4)
    Numberofrows = int(math.ceil(round((int(EndingAddress, 16) - int(StartingAddress, 16)) / 5.0) * 5.0) / 16)
    RowNumber = {}
    InitialCondition = ["00", "00", "00", "00", "00", "00", "00", "00", "00", "00", "00", "00", "00", "00", "00", "00"]
    # Creation of Pandas DataFrame
    for i in range(0, int(Numberofrows / 10) + 1):
        RowNumber[(hex(int(StartingAddress, 16) + int(16 * i))[2:].zfill(4))] = InitialCondition
    df = pd.DataFrame(RowNumber,
                      columns=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f'],
                      index=list(RowNumber.keys()))
    df = df.fillna("00")
    # Processing our TRecord
    for TR in TRecord:
        Bytes = ""
        BytesList = []
        TRecordStart = hex(int(TR[1], 16))
        for j in range(3, len(TR)):
            Bytes += TR[j]
        BytesList = re.findall('..', Bytes)
        itr = 0
        for Byte in BytesList:
            currentplace = hex(int(TRecordStart, 16) + itr)[2:].zfill(4)
            columnnumber = hex(int(currentplace, 16) % 16)[2:]
            currentrow = hex(int(currentplace, 16) - int(columnnumber, 16))[2:].zfill(4)
            itr += 1
            df.at[currentrow, columnnumber] = Byte
    # Creation of txt File
    generate_table(df)
    pd.options.display.max_columns = 16
    df["First"] = df['0'] + df['1'] + df['2'] + df['3']
    df["Second"] = df['4'] + df['5'] + df['6'] + df['7']
    df["Third"] = df['8'] + df['9'] + df['a'] + df['b']
    df["Fourth"] = df['c'] + df['d'] + df['e'] + df['f']
    df.drop(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f'], axis=1, inplace=True)
    df.to_csv("AbsoluteLoader.csv", sep=" ", header=False)


def LinkingLoader():
    OSStartingAddress = input(
        "Please Enter The Address Provided by the operating system : ")  # Taking Input which is the address chosen by operating systems to load out HTE Record
    StartingAddress = OSStartingAddress
    with open('input.txt') as my_file:  # Parsing the input file provided by the user
        HTE = [line.split('.') for line in my_file]
    ProgramLengths = {}
    EXSTB = {}
    DRecord = {}
    for line in HTE:
        if line[0] == 'H':  # H Record Extraction
            ProgramLengths[line[1]] = line[3]
            ProgramNames = list(ProgramLengths.keys())
        if line[0] == 'D':  # D Record Extraction
            DRecord[ProgramNames[-1]] = line[1:]
    # Creation of first half of the External Symbol Table
    for i in range(0, len(ProgramNames)):
        if i == 0:
            EXSTB[ProgramNames[0]] = hex(int(OSStartingAddress, 16))[2:].zfill(5)
        else:
            EXSTB[ProgramNames[i]] = hex(
                int(EXSTB[ProgramNames[i - 1]], 16) + int(ProgramLengths[ProgramNames[i - 1]], 16))[2:].zfill(5)
            EndingAddress = hex(int(EXSTB[ProgramNames[i]], 16) + int(ProgramLengths[ProgramNames[i]], 16))[2:].zfill(4)
    # Creation of second half of the External Symbol Table
    for ProgramName in ProgramNames:
        for i in range(0, len(DRecord[ProgramName]), 2):
            EXSTB[DRecord[ProgramName][i]] = hex(int(DRecord[ProgramName][i + 1], 16) + int(EXSTB[ProgramName], 16))[2:]
    TRecord = {}
    RowNumber = {}
    itt = 0
    # Extraction of TRecord Related to each Program
    for ProgramName in ProgramNames:
        TRecord[ProgramName] = []
        while (itt < len(HTE)):
            if HTE[itt][0] == 'E':
                itt += 1
                break
            elif HTE[itt][0] == 'T':
                TRecord[ProgramName].append(HTE[itt])
            itt += 1
    Numberofrows = int(round((int(EndingAddress, 16) - int(StartingAddress, 16)) / 5.0) * 5.0)
    print(StartingAddress, EndingAddress, Numberofrows)
    InitialCondition = ["00", "00", "00", "00", "00", "00", "00", "00", "00", "00", "00", "00", "00", "00", "00", "00"]
    for i in range(0, int(Numberofrows / 16) + 1):
        RowNumber[(hex(int(StartingAddress, 16) + int(16 * i))[2:].zfill(4))] = InitialCondition
    df = pd.DataFrame(RowNumber,
                      columns=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f'],
                      index=list(RowNumber.keys()))
    df = df.fillna("00")
    # Processing TRecords for each of the designated HTE Records
    for ProgramName in ProgramNames:
        TR = TRecord[ProgramName]
        for T in TR:
            Bytes = ""
            BytesList = []
            TRecordStart = hex(int(T[1], 16) + int(EXSTB[ProgramName], 16))
            for j in range(3, len(T)):
                Bytes += T[j]
            BytesList = re.findall('..', Bytes)
            itr = 0
            for Byte in BytesList:
                currentplace = hex(int(TRecordStart, 16) + itr)[2:].zfill(4)
                columnnumber = hex(int(currentplace, 16) % 16)[2:]
                currentrow = hex(int(currentplace, 16) - int(columnnumber, 16))[2:].zfill(4)
                itr += 1
                df.at[currentrow, columnnumber] = Byte
    MRecord = {}
    itt = 0
    # Extracting MRecord for each program
    for ProgramName in ProgramNames:
        MRecord[ProgramName] = []
        while (itt < len(HTE)):
            if HTE[itt][0] == 'E':
                itt += 1
                break
            elif HTE[itt][0] == 'M':
                MRecord[ProgramName].append(HTE[itt])
            itt += 1
    # Processing MRecords for each of the designated HTE Records
    for ProgramName in ProgramNames:
        MR = MRecord[ProgramName]
        for M in MR:
            MRecordStart = hex(int(M[1], 16) + int(EXSTB[ProgramName], 16))[2:].zfill(6)
            currentplace = hex(int(MRecordStart, 16))[2:].zfill(4)
            columnnumber = hex(int(currentplace, 16) % 16)[2:]
            currentrow = hex(int(currentplace, 16) - int(columnnumber, 16))[2:].zfill(4)
            BytesSequence = df.loc[currentrow, columnnumber] + df.loc[currentrow][hex(int(columnnumber, 16) + 1)[2:]] + \
                            df.loc[currentrow][hex(int(columnnumber, 16) + 2)[2:]]
            if (M[2][0:2] == '05'):  # Case 1 : MRecord Related to an Instruction
                if (M[2][2] == '+'):
                    RequiredModification = int(EXSTB[M[2][3:-1]], 16)
                elif (M[2][2] == '-'):
                    RequiredModification = int(EXSTB[M[2][3:-1]], 16) * -1
                NonModifiableCharacter = df.loc[currentrow, columnnumber][0]
                Modified = hex(int(BytesSequence[1:], 16) + RequiredModification)[2:].zfill(6)
                Modified = NonModifiableCharacter + Modified[1:]
            elif (M[2][0:2] == '06'):  # Case 1 : MRecord Related to a Reservation
                if (M[2][2] == '+'):
                    RequiredModification = int(EXSTB[M[2][3:-1]], 16)
                elif (M[2][2] == '-'):
                    RequiredModification = int(EXSTB[M[2][3:-1]], 16) * -1
                Modified = hex(int(BytesSequence, 16) + RequiredModification)[2:].zfill(6)
            Bytes = ""
            BytesList = []
            BytesList = re.findall('..', Modified)
            for i in range(0, 3):
                df.loc[currentrow][hex(int(columnnumber, 16) + i)[2:]] = BytesList[i]
    # Creation of txt File
    display(df)
    pd.options.display.max_columns = 16
    df["First"] = df['0'] + df['1'] + df['2'] + df['3']
    df["Second"] = df['4'] + df['5'] + df['6'] + df['7']
    df["Third"] = df['8'] + df['9'] + df['a'] + df['b']
    df["Fourth"] = df['c'] + df['d'] + df['e'] + df['f']
    df.drop(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f'], axis=1, inplace=True)
    df.to_csv("LinkingLoader.csv", sep=" ", header=False)


option = int(input("Please Specify your mode of Operation : "))
print("Please note that your HTE Record should be provided in the input.txt file")
if (option == 0):
    AbsoluteLoader()
elif (option == 1):
    LinkingLoader()
print("Thank you for using the program.")

Please Specify your mode of Operation : 1
Please note that your HTE Record should be provided in the input.txt file
Please Enter The Address Provided by the operating system : 4020
4020 41b7 405


,0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f
4020,6F,20,1D,75,01,2C,05,00,00,03,90,40,88,2B,A0,13
4030,37,20,03,0F,20,0D,90,41,A0,15,3B,2F,EC,00,00,00
4040,00,00,00,00,41,b4,03,10,41,b4,6F,10,40,43,9C,40
4050,0C,10,40,5e,98,40,6F,20,08,94,04,FB,20,05,00,00
4060,00,00,00,00,00,00,00,80,00,6D,00,03,75,01,2C,05
4070,00,00,03,A0,13,2B,21,3C,2B,30,40,7f,0F,21,35,90
4080,41,A0,15,3B,2F,EA,00,00,00,00,00,00,00,00,00,00
4090,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00
40a0,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00
40b0,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00,00


Thank you for using the program.
